# PyCity Schools Analysis

- On average, students in the district tend to do better at reading than at math, which is evident from both the average scores for each exam as well as the percentage of students that passed each exam. The average reading test score is 81.88, compared to 78.99 for math. ~86% of the students in the district passed the reading test, while ~75% passed the math test. 

- Schools that spend a higher budget per student do not have higher overall test results. In fact, the schools with the highest overall average test scores are the ones that have the lowest budget per student (<585 dollars).

- Small and medium sized schools outperform large sized schools on overall passing performance (90-91% passing vs. 58%). This is largely driven by a low % of students passing math exams at large schools (~70%), compared to >90% at small and medium sized schools.

In [1]:
# Import dependencies
import pandas as pd
 
# Store files into Pandas DataFrames
schoolData = pd.read_csv("Resources/schools_complete.csv")
studentData = pd.read_csv("Resources/students_complete.csv")

# Combine the data into a single dataFrame. 
districtData = pd.merge(studentData, schoolData, how="left", on=["school_name", "school_name"])

## District Summary

In [2]:
# Calculate the Totals (Schools and Students)
totalSchoolCount = len(districtData["school_name"].unique())
totalStudentCount = len(districtData["Student ID"])

# Calculate the Total Budget
totalBudget = schoolData["budget"].sum()

In [3]:
# Calculate the Average Scores
avgMath = districtData["math_score"].mean()
avgReading = districtData ["reading_score"].mean()

In [4]:
# Calculate the Percentage Pass Rates
passedReadingCount = districtData[districtData["reading_score"] > 69]
passedMathCount = districtData[districtData["math_score"] > 69]
passedBothCount = districtData[(districtData["reading_score"] > 69) & (districtData["math_score"] > 69)]

passedReading = (len(passedReadingCount) / float(totalStudentCount))*100
passedMath = (len(passedMathCount) / float(totalStudentCount))*100
passedBoth = (len(passedBothCount) / float(totalStudentCount))*100

In [5]:
# Make a district summary DataFrame using a list of Dictionaries
districtSummary = [
    {"Total Schools": totalSchoolCount, "Total Students": totalStudentCount,
     "Total Budget": totalBudget, "Average Math Score": avgMath,
     "Average Reading Score": avgReading, "% Passing Math": passedMath, 
     "% Passing Reading": passedReading, "% Overall Passing": passedBoth}
]
districtSummaryDF = pd.DataFrame(districtSummary)

# Using the "map" function to format the Total Students and Total Budget column
districtSummaryDF["Total Students"] = districtSummaryDF["Total Students"].map("{:,}".format)
districtSummaryDF["Total Budget"] = districtSummaryDF["Total Budget"].map("${:,}".format)
districtSummaryDF["Average Math Score"] = districtSummaryDF["Average Math Score"].map("{:.2f}".format)
districtSummaryDF["Average Reading Score"] = districtSummaryDF["Average Reading Score"].map("{:.2f}".format)
districtSummaryDF["% Passing Math"] = districtSummaryDF["% Passing Math"].map("{:.2f}%".format)
districtSummaryDF["% Passing Reading"] = districtSummaryDF["% Passing Reading"].map("{:.2f}%".format)
districtSummaryDF["% Overall Passing"] = districtSummaryDF["% Overall Passing"].map("{:.2f}%".format)

# Display the Data Frame
districtSummaryDF

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",78.99,81.88,74.98%,85.81%,65.17%


## School Summary

In [6]:
# change the index of the school data to school names
updatedSchoolData = schoolData.set_index("school_name")

# Determine the School Type
schoolType = updatedSchoolData["type"]

# Calculate the total student count
studentCount = updatedSchoolData["size"]

# Calculate the total school budget 
schoolBudget = updatedSchoolData["budget"]

# Calculate per student budget and add it to dataframe
perStudentBudget = schoolBudget/studentCount
updatedSchoolData["Per Student Budget"] = perStudentBudget

# Calculate the average test scores 

# Use a groupbyfunction 
groupedStudentData = studentData.groupby(["school_name"])

# Calculate the average reading and math scores
avgSchoolReading = groupedStudentData["reading_score"].mean()
avgSchoolMath = groupedStudentData["math_score"].mean()

In [7]:
# Get the students who passed math and passed reading by creating separate filtered DataFrames.
passedReadingDF = studentData[studentData["reading_score"] > 69]
passedMathDF = studentData[studentData["math_score"] > 69]
passedBothDF = studentData[(studentData["reading_score"] > 69) & (studentData["math_score"] > 69)]

# Calculate percentage pass rates 

# Use a groupbyfunction for the 3 data frames
groupedReadingData = passedReadingDF.groupby(["school_name"])
groupedMathData = passedMathDF.groupby(["school_name"])
groupedBothData = passedBothDF.groupby(["school_name"])

# Calculate the percentage pass rates
passedReadingPct = (groupedReadingData["Student ID"].count()/studentCount)*100
passedMathPct = (groupedMathData["Student ID"].count()/studentCount)*100
passedBothPct = (groupedBothData["Student ID"].count()/studentCount)*100

In [8]:
# Convert to DataFrame --> this DF will be used later
schoolSummaryNumericDF = pd.DataFrame({
    "School Type": schoolType, "Total Students": studentCount,
    "Total School Budget": schoolBudget, "Per Student Budget": perStudentBudget,
    "Average Math Score": avgSchoolMath, "Average Reading Score": avgSchoolReading,
    "% Passing Math": passedMathPct, "% Passing Reading": passedReadingPct,
    "% Overall Passing": passedBothPct
})

In [9]:
# format the output using Map
schoolSummaryDF = schoolSummaryNumericDF.rename_axis("") # changing the name of the index column to nothing
schoolSummaryDF["Total Students"] = schoolSummaryNumericDF["Total Students"].map("{:,}".format)
schoolSummaryDF["Total School Budget"] = schoolSummaryNumericDF["Total School Budget"].map("${:,}".format)
schoolSummaryDF["Per Student Budget"] = schoolSummaryNumericDF["Per Student Budget"].map("${:.0f}".format)
schoolSummaryDF["Average Math Score"] = schoolSummaryNumericDF["Average Math Score"].map("{:.2f}".format)
schoolSummaryDF["Average Reading Score"] = schoolSummaryNumericDF["Average Reading Score"].map("{:.2f}".format)
schoolSummaryDF["% Passing Math"] = schoolSummaryNumericDF["% Passing Math"].map("{:.2f}%".format)
schoolSummaryDF["% Passing Reading"] = schoolSummaryNumericDF["% Passing Reading"].map("{:.2f}%".format)
schoolSummaryDF["% Overall Passing"] = schoolSummaryNumericDF["% Overall Passing"].map("{:.2f}%".format)

# Display the DataFrame
schoolSummaryDF

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928",$628,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916",$644,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020",$652,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087",$581,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06%,81.22%,53.54%


## Top Performing Schools (By % Overall Passing)

In [10]:
# Sort and show top five schools
top5Schools = schoolSummaryDF.sort_values("% Overall Passing", ascending=False)
top5Schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130",$638,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574",$578,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

In [11]:
# Sort and show bottom five schools
bottom5Schools = schoolSummaryDF.sort_values("% Overall Passing", ascending=True)
bottom5Schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363",$637,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020",$652,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06%,81.22%,53.54%


## Math Scores by Grade

In [12]:
# Create data series of scores by grade levels 
ninthGraders = studentData[studentData["grade"] == "9th"]
tenthGraders = studentData[studentData["grade"] == "10th"]
eleventhGraders = studentData[studentData["grade"] == "11th"]
twelfthGraders = studentData[studentData["grade"] == "12th"]

# Group each by school name
grouped9thMath = ninthGraders.groupby(["school_name"])["math_score"].mean()
grouped10thMath = tenthGraders.groupby(["school_name"])["math_score"].mean()
grouped11thMath = eleventhGraders.groupby(["school_name"])["math_score"].mean()
grouped12thMath = twelfthGraders.groupby(["school_name"])["math_score"].mean()

# Combine into a single data frame 
mathScoresDF = pd.DataFrame({
    "9th": grouped9thMath, 
    "10th": grouped10thMath,
    "11th": grouped11thMath, 
    "12th": grouped12thMath
})

# Using Map to format DataFrame
mathScoresDF = mathScoresDF.rename_axis("") # changing the name of the index column to nothing
mathScoresDF["9th"] = mathScoresDF["9th"].map("{:.2f}".format)
mathScoresDF["10th"] = mathScoresDF["10th"].map("{:.2f}".format)
mathScoresDF["11th"] = mathScoresDF["11th"].map("{:.2f}".format)
mathScoresDF["12th"] = mathScoresDF["12th"].map("{:.2f}".format)

# Display the DataFrame
mathScoresDF


,9th,10th,11th,12th
,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

In [13]:
# using the data series from the above exercise, group each by school name
grouped9thReading = ninthGraders.groupby(["school_name"])["reading_score"].mean()
grouped10thReading = tenthGraders.groupby(["school_name"])["reading_score"].mean()
grouped11thReading = eleventhGraders.groupby(["school_name"])["reading_score"].mean()
grouped12thReading = twelfthGraders.groupby(["school_name"])["reading_score"].mean()

# Combine into a single data frame 
readingScoresDF = pd.DataFrame({
    "9th": grouped9thReading, 
    "10th": grouped10thReading,
    "11th": grouped11thReading, 
    "12th": grouped12thReading
})

# Using Map to format DataFrame
readingScoresDF = readingScoresDF.rename_axis("") # changing the name of the index column to nothing
readingScoresDF["9th"] = readingScoresDF["9th"].map("{:.2f}".format)
readingScoresDF["10th"] = readingScoresDF["10th"].map("{:.2f}".format)
readingScoresDF["11th"] = readingScoresDF["11th"].map("{:.2f}".format)
readingScoresDF["12th"] = readingScoresDF["12th"].map("{:.2f}".format)

# Display the DataFrame
readingScoresDF

,9th,10th,11th,12th
,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

In [14]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Establish the bins 
bins = [0, 585, 630, 645, 1000]
groupNames = ["<$585", "$585-630", "$630-645", "$645+"]

# Create the new column based on the bins
# We'll use the numeric version of the schoolSummary DF
schoolSummaryNumericDF["Spending Ranges (Per Student)"] = \
pd.cut(schoolSummaryNumericDF["Per Student Budget"], bins, labels = groupNames)

In [15]:
# Calculate averages for the desired columns using groupby 
spendingAvgMath = schoolSummaryNumericDF.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spendingAvgReading = schoolSummaryNumericDF.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spendingAvgMathPassing = schoolSummaryNumericDF.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spendingAvgReadingPassing = schoolSummaryNumericDF.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
spendingAvgBothPassing = schoolSummaryNumericDF.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()


In [16]:
# Assemble into DataFrame
schoolSpendingDF = pd.DataFrame({
    "Average Math Score": spendingAvgMath, 
    "Average Reading Score": spendingAvgReading,
    "% Passing Math": spendingAvgMathPassing, 
    "% Passing Reading": spendingAvgReading,
    "% Overall Passing": spendingAvgBothPassing
})

# Format DataFrame using map
schoolSpendingDF["Average Math Score"] = schoolSpendingDF["Average Math Score"].map("{:.2f}".format)
schoolSpendingDF["Average Reading Score"] = schoolSpendingDF["Average Reading Score"].map("{:.2f}".format)
schoolSpendingDF["% Passing Math"] = schoolSpendingDF["% Passing Math"].map("{:.2f}%".format)
schoolSpendingDF["% Passing Reading"] = schoolSpendingDF["% Passing Reading"].map("{:.2f}%".format)
schoolSpendingDF["% Overall Passing"] = schoolSpendingDF["% Overall Passing"].map("{:.2f}%".format)

# Display the DataFrame
schoolSpendingDF

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46%,83.93%,90.37%
$585-630,81.90,83.16,87.13%,83.16%,81.42%
$630-645,78.52,81.62,73.48%,81.62%,62.86%
$645+,77.00,81.03,66.16%,81.03%,53.53%


## Scores by School Size

In [22]:
# Establish the bins.
bins2 = [0, 1000, 2000, 5000]
groupNames2 = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Categorize the spending based on the bins 
# Use the numeric DF (schoolSummaryNumericDF)
schoolSummaryNumericDF["School Size"] = \
pd.cut(schoolSummaryNumericDF["Total Students"], bins2, labels = groupNames2)

In [18]:
# Calculate averages for the desired columns. 
sizeAvgMath = schoolSummaryNumericDF.groupby(["School Size"])["Average Math Score"].mean()
sizeAvgReading = schoolSummaryNumericDF.groupby(["School Size"])["Average Reading Score"].mean()
sizeAvgMathPassing = schoolSummaryNumericDF.groupby(["School Size"])["% Passing Math"].mean()
sizeAvgReadingPassing = schoolSummaryNumericDF.groupby(["School Size"])["% Passing Reading"].mean()
sizeAvgBothPassing = schoolSummaryNumericDF.groupby(["School Size"])["% Overall Passing"].mean()

In [19]:
# Assemble into DataFrame
schoolSizeDF = pd.DataFrame({
    "Average Math Score": sizeAvgMath, 
    "Average Reading Score": sizeAvgReading,
    "% Passing Math": sizeAvgMathPassing, 
    "% Passing Reading": sizeAvgReadingPassing,
    "% Overall Passing": sizeAvgBothPassing
})

# Format Data Frame Using Map
schoolSizeDF["Average Math Score"] = schoolSizeDF["Average Math Score"].map("{:.2f}".format)
schoolSizeDF["Average Reading Score"] = schoolSizeDF["Average Reading Score"].map("{:.2f}".format)
schoolSizeDF["% Passing Math"] = schoolSizeDF["% Passing Math"].map("{:.2f}%".format)
schoolSizeDF["% Passing Reading"] = schoolSizeDF["% Passing Reading"].map("{:.2f}%".format)
schoolSizeDF["% Overall Passing"] = schoolSizeDF["% Overall Passing"].map("{:.2f}%".format)

# Display results
schoolSizeDF


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,89.88%
Medium (1000-2000),83.37,83.86,93.60%,96.79%,90.62%
Large (2000-5000),77.75,81.34,69.96%,82.77%,58.29%


## Scores by School Type

In [20]:
# Create new series using groupby 
# Calculate averages for the desired columns. 
typeAvgMath = schoolSummaryNumericDF.groupby(["School Type"])["Average Math Score"].mean()
typeAvgReading = schoolSummaryNumericDF.groupby(["School Type"])["Average Reading Score"].mean()
typeAvgMathPassing = schoolSummaryNumericDF.groupby(["School Type"])["% Passing Math"].mean()
typeAvgReadingPassing = schoolSummaryNumericDF.groupby(["School Type"])["% Passing Reading"].mean()
typeAvgBothPassing = schoolSummaryNumericDF.groupby(["School Type"])["% Overall Passing"].mean()

In [21]:
# Assemble into DataFrame
schoolTypeDF = pd.DataFrame({
    "Average Math Score": typeAvgMath, 
    "Average Reading Score": typeAvgReading,
    "% Passing Math": typeAvgMathPassing, 
    "% Passing Reading": typeAvgReadingPassing,
    "% Overall Passing": typeAvgBothPassing
})

# Format Data Frame Using Map
schoolTypeDF["Average Math Score"] = schoolTypeDF["Average Math Score"].map("{:.2f}".format)
schoolTypeDF["Average Reading Score"] = schoolTypeDF["Average Reading Score"].map("{:.2f}".format)
schoolTypeDF["% Passing Math"] = schoolTypeDF["% Passing Math"].map("{:.2f}%".format)
schoolTypeDF["% Passing Reading"] = schoolTypeDF["% Passing Reading"].map("{:.2f}%".format)
schoolTypeDF["% Overall Passing"] = schoolTypeDF["% Overall Passing"].map("{:.2f}%".format)

# Display results
schoolTypeDF

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,90.43%
District,76.96,80.97,66.55%,80.80%,53.67%
